In [1]:
def test_method(self): 
    print("This is Test class method!") 
  
# creating a base class  
class Base: 
    def myfun(self): 
        print("This is inherited method!") 
  

In [2]:
# Creating Test class dynamically using 
# type() method directly 
Test = type('Test', (Base, ), dict(x="atul", my_method=test_method)) 

In [3]:
help(Test) #parent of Test class in Base, instance variable is atul, and have my_method

Help on class Test in module __main__:

class Test(Base)
 |  Method resolution order:
 |      Test
 |      Base
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  my_method = test_method(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  x = 'atul'
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Base:
 |  
 |  myfun(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Base:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [6]:
type(Test)

type

In [4]:
test_obj = Test()

In [5]:
type(test_obj)

__main__.Test

In [8]:
test_obj.x

'atul'

In [9]:
test_obj.my_method()

This is Test class method!


# Restrict Inheritance to one level

In [10]:
# our metaclass 
class MultiBases(type): 
    # overriding __new__ method 
    def __new__(cls, clsname, bases, clsdict): 
        # if no of base classes is greator than 1 
        # raise error 
        if len(bases)>1: 
            raise TypeError("Inherited multiple base classes!!!") 
          
        # else execute __new__ method of super class, ie. 
        # call __init__ of type class 
        return super().__new__(cls, clsname, bases, clsdict) 

In [11]:
# metaclass can be specified by 'metaclass' keyword argument 
# now MultiBase class is used for creating classes 
# this will be propagated to all subclasses of Base 
class Base(metaclass=MultiBases): 
    pass

In [12]:
# no error is raised 
class A(Base): 
    pass

In [13]:
# no error is raised 
class B(Base): 
    pass

In [15]:
# This will raise an error! 
class C(A, B): 
    pass

TypeError: Inherited multiple base classes!!!

# Solving problem with metaclass



In [16]:
from functools import wraps 
from pprint import pprint

In [25]:
vars(Calc)

mappingproxy({'__module__': '__main__',
              'add': <function __main__.Calc.add(self, x, y)>,
              'mul': <function __main__.Calc.mul(self, x, y)>,
              'div': <function __main__.Calc.div(self, x, y)>,
              '__dict__': <attribute '__dict__' of 'Calc' objects>,
              '__weakref__': <attribute '__weakref__' of 'Calc' objects>,
              '__doc__': None})

In [17]:
def debug(func): 
    '''decorator for debugging passed function'''
      
    @wraps(func) 
    def wrapper(*args, **kwargs): 
        print("Full name of this method:", func.__qualname__) 
        return func(*args, **kwargs) 
    return wrapper 
  
def debugmethods(cls): 
    '''class decorator make use of debug decorator 
       to debug class methods '''
      
    # check in class dictionary for any callable(method) 
    # if exist, replace it with debugged version 
    for key, val in vars(cls).items(): 
        if callable(val): # if value is callable like function
            setattr(cls, key, debug(val))  # then add decorator debug on it (simply replacing it with debug(val))
    return cls


In [18]:
# sample class 
@debugmethods
class Calc: 
    def add(self, x, y): 
        return x+y 
    def mul(self, x, y): 
        return x*y 
    def div(self, x, y): 
        return x/y 

In [19]:
mycal = Calc() 
print(mycal.add(2, 3)) 
print(mycal.mul(5, 2))

Full name of this method: Calc.add
5
Full name of this method: Calc.mul
10


## Metaclass based solution solved all above issue

In [27]:
from functools import wraps 
  
def debug(func): 
    '''decorator for debugging passed function'''
      
    @wraps(func) 
    def wrapper(*args, **kwargs): 
        print("Full name of this method:", func.__qualname__) 
        return func(*args, **kwargs) 
    return wrapper 
  
def debugmethods(cls): 
    '''class decorator make use of debug decorator 
       to debug class methods '''
      
    for key, val in vars(cls).items(): 
        if callable(val): 
            setattr(cls, key, debug(val)) 
    return cls

In [28]:
# Metaclass
class debugMeta(type): 
    '''meta class which feed created class object 
       to debugmethod to get debug functionality 
       enabled objects'''
      
    def __new__(cls, clsname, bases, clsdict): 
        obj = super().__new__(cls, clsname, bases, clsdict) 
        obj = debugmethods(obj) #added decorator debugmethods for all class inheritted from this metaclass
        return obj 


In [29]:
# base class with metaclass 'debugMeta' 
# now all the subclass of this  
# will have debugging applied 
class Base(metaclass=debugMeta):pass
  
# inheriting Base 
class Calc(Base): 
    def add(self, x, y): 
        return x+y 
      
# inheriting Calc 
class Calc_adv(Calc): 
    def mul(self, x, y): 
        return x*y 

In [30]:
# Now Calc_adv object showing 
# debugging behaviour 
mycal = Calc_adv() 
print(mycal.mul(2, 3)) 

Full name of this method: Calc_adv.mul
6
